# Cirrus Demo

## Simple Example

This will run a simple logistic regression on the Criteo Kaggle dataset

In [1]:
import cirrus
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import sys
print(sys.version)

print __version__ # requires version >= 1.9.0

2.7.12 (default, Dec  4 2017, 14:50:18) 
[GCC 5.4.0 20160609]
2.7.0


In [2]:

import numpy as np 
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

init_notebook_mode(connected=True)
stream_ids = tls.get_credentials_file()['stream_ids']

# Get stream id from stream id list 
stream_id = stream_ids[0]


py.iplot([{'x': [], 'y': [], 'type': 'scatter', 'mode': 'lines+markers',
            'stream': {'token': stream_id, 'maxpoints': 80}
          }],
        filename='Time Series', fileopt='overwrite')

In [3]:

# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s = py.Stream(stream_id)

# We then open a connection
s.open()

In [4]:
def progress_callback(time_loss, cost, task):
    print("Current training loss:", time_loss, "current cost ($): ", cost)
    s.write(dict(x=time_loss[0], y=time_loss[1]))



In [6]:
data_bucket = 'cirrus-criteo-kaggle-19b-random'
model = 'model_v1'

lr_task = cirrus.LogisticRegression(
             # number of workers
             n_workers = 30,
             # number of parameter servers
             n_ps = 2,
             # worker size in MB
             worker_size = 128,
             # path to s3 bucket with input dataset
             dataset = data_bucket,
             # sgd update LR and epsilon
             learning_rate=0.01,
             epsilon=0.0001,
             progress_callback = progress_callback,
             # stop workload after these many seconds
             timeout = 0,
             # stop workload once we reach this loss
             threshold_loss=0,
             # resume execution from model stored in this s3 bucket
             resume_model = model,
             # aws key name
             key_name='mykey',
             # path to aws key
             key_path='/home/camus/mykey.pem',
             # ip where ps lives
             ps_ip_public='ec2-18-237-250-161.us-west-2.compute.amazonaws.com',
             ps_ip_private='172.31.10.1',
             # username of VM
             ps_username='ubuntu',
             # choose between adagrad, sgd, nesterov, momentum
             opt_method = 'adagrad',
             # checkpoint model every x secs
             checkpoint_model = 60, 
             # 
             minibatch_size=20,
             # model size
             model_bits=19,
             # whether to filter gradient weights
             use_grad_threshold=False,
             # threshold value
             grad_threshold=0.001,
             # range of training minibatches
             train_set=(0,824),
             # range of testing minibatches
             test_set=(835,840)
             )

lr_task.run()

#model, loss = lr_task.wait()

Running Logistic Regression workload
Starting LogisticRegressionTask
User's specific ip: ec2-18-237-250-161.us-west-2.compute.amazonaws.com
Copying ps to vm..
Done waiting... Attempting to copy over binary
Copied PS binary to VM
Defining configuration file

input_path: /mnt/efs/criteo_kaggle/train.csv 
input_type: csv
num_classes: 2 
num_features: 13 
limit_cols: 14 
normalize: 1 
limit_samples: 50000000 
s3_size: 50000 
use_bias: 1 
model_type: LogisticRegression 
minibatch_size: 20 
learning_rate: 0.010000 
epsilon: 0.000100 
model_bits: 19 
s3_bucket: cirrus-criteo-kaggle-19b-random 
use_grad_threshold: 0 
grad_threshold: 0.001000 
train_set: 0-824 
test_set: 835-840

Launching ps
Launching parameter server
('cmd:', 'ssh -o "StrictHostKeyChecking no" -i /home/camus/mykey.pem ubuntu@ec2-18-237-250-161.us-west-2.compute.amazonaws.com "nohup ./parameter_server --config config_lr.txt --nworkers 10000 --rank 1 &> ps_output &"')
Launching lambdas
Starting error taskLambdas have been launc

In [ ]:
s.close